In [1]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
import numpy as np
#from xgboost import XGBClassifier
from sklearn.linear_model import LinearRegression, LassoCV, Lasso, RidgeCV, ElasticNetCV
from sklearn.kernel_ridge import KernelRidge

from sklearn.ensemble import RandomForestRegressor
from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, mean_squared_error
from modelling_functions_new import *

## Processing for Models

In [2]:
#### Read Data files
dict_dictonary = {}
housing, housing_features, feat_labels = read_and_clean(filepath = "../data/clean_train.csv")

['LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'X1stFlrSF', 'X2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'X3SsnPorch', 'ScreenPorch', 'PoolArea', 'SalePrice']


In [3]:
htest_id, htest_features, htest_labels = read_and_clean(filepath = "../data/clean_test.csv", test = True)

['LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'X1stFlrSF', 'X2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'X3SsnPorch', 'ScreenPorch', 'PoolArea']


In [ ]:
training = housing_features
testing = htest_features

In [ ]:
print(training.shape)
print(testing.shape)
housing.saleprice[1:2]

In [ ]:
test_col = testing.columns
train_col = training.columns

missing = [x for x in train_col if x not in test_col]
needed = [ x for x in test_col if x not in train_col]


In [ ]:
training = training.drop(missing, axis=1)
testing = testing.drop(needed, axis=1)

In [ ]:
htrain, htest, ptrain, ptest = train_test_split(training, housing.saleprice, test_size = 0.2)

## Lasso

In [ ]:
alp = optimize_penalty(htrain, ptrain, model=Lasso, min_=1e-4, max_=10, step=0.0001, plot=True)

In [ ]:
lasso = Lasso(normalize=False)
lasso.set_params(alpha = 3e-4)
lasso.fit(htrain, ptrain)

print(lasso.score(htrain, ptrain))
print(lasso.score(htest, ptest))
#print(selected_coefs)
pred_lasso = lasso.predict(htest)
mean_squared_error(pred_lasso, ptest)

In [ ]:
pred = np.exp(lasso.predict(testing))

In [ ]:
Submission(htest_id, pred, "lasso.csv")

In [ ]:
drop_col = list(training.columns[np.where(lasso.coef_ == 0)[0]])
print(drop_col)

In [ ]:
training = training.drop(drop_col, axis = 1)
testing = testing.drop(drop_col, axis = 1)

In [ ]:
print(training.shape)
print(testing.shape)

In [ ]:
htrain, htest, ptrain, ptest = train_test_split(training, housing.saleprice, test_size = 0.2, random_state = 480)

## RandomForestRegressor

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 10000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 200, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [15, 20, 30, 40]
# Minimum number of samples required at each leaf node
min_samples_leaf = [16, 32, 64]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=0, n_jobs = -1)
# Fit the random search model
rf_random.fit(htrain, ptrain)

In [ ]:
rf_random.best_params_

In [ ]:
#rf_random.fit(htrain, ptrain)

print(rf_random.score(htrain, ptrain))
pred_rf_random = rf_random.predict(htest)
mean_squared_error(pred_rf_random, ptest)

In [ ]:
clf = RandomForestRegressor(n_estimators=10000, 
                            random_state=9, 
                            n_jobs=-1, 
                            min_samples_split = 40, 
                            min_samples_leaf = 1, 
                            max_features = 'sqrt', 
                            max_depth = 162,
                            bootstrap = True)
#sfm = SelectFromModel(clf, threshold = 0.01)
#sfm.fit(htrain, ptrain)
# ## Not Finished
# # Measure Feature Importance
#feature_selected = []
#for feature_list_index in sfm.get_support(indices=True):
#     feature_selected.append(feat_labels[feature_list_index])
#proxy = feature_selected
#trimmed = ['x1stflrsf', 'x2ndflrsf', 'garagecars', 'overallcond', 'saleprice', 'Unnamed: 0', 'bsmtfinsf1']
#testing = testing[proxy]
#housing_features = housing_features[trimmed_features]
#print(feature_selected)

In [ ]:
clf.fit(htrain, ptrain)

clf.score(htest, ptest)

In [ ]:
pred_clf = clf.predict(htest)
mean_squared_error(pred_clf, ptest)

## ElasticNet

In [ ]:
regr = ElasticNetCV(l1_ratio = [.1, .2, .3, .4, .5, .6, .7, .8, .9, .99, 1], 
                    cv = 5, 
                    n_jobs = -1,
                    normalize = True,
                    verbose = 0)

In [ ]:
regr.fit(htrain, ptrain)

In [ ]:
regr.score(htest, ptest)
pred_regr = regr.predict(htest)
mean_squared_error(pred_regr, ptest)

In [ ]:
regr.l1_ratio_

### Ridge

In [ ]:
ridge = RidgeCV(normalize = True, cv = 5)

ridge.fit(htrain, ptrain)

In [ ]:
print(ridge.score(htest, ptest))
pred_ridge = ridge.predict(htest)
mean_squared_error(pred_ridge, ptest)

In [ ]:
t = pd.read_csv('submission_13703.csv')

In [ ]:
lm = LinearRegression(normalize = True)
lm.fit(htrain, ptrain)
print(lm.score(htest, ptest))
pred_linear = lm.predict(htest)
mean_squared_error(pred_linear, ptest)

In [ ]:
a = 0.1
b = 0.3
c = 0.5
d = 0.1

pred_ridge_sub = ridge.predict(testing)
pred_lasso_sub = lasso.predict(testing)
pred_linear_sub = lm.predict(testing)
pred_regr_sub = regr.predict(testing)
#pred_rf_random_sub = rf_random.predict(testing)

pred_sum = np.exp(a*(pred_ridge_sub) + b*(pred_lasso_sub) + c*(pred_regr_sub) + d*(pred_linear_sub))

Submission(htest_id, pred_sum, "stacked_submission.csv")

In [ ]:
ridge.predict(testing)

In [ ]:
pred_sum.shape

In [ ]:
a = 0.1
b = 0.3
c = 0.5
d = 0.1



pred_sum = a*(pred_ridge) + b*(pred_lasso) + c*(pred_regr) + d*(pred_linear)
mean_squared_error(pred_sum, ptest)

In [ ]:
pred_sum = (pred_ridge + pred_lasso + pred_regr + pred_linear)/4
mean_squared_error(pred_sum, ptest)

In [ ]:
mean_squared_error(pred_sum, ptest)

## Kernal Ridge

In [ ]:
from sklearn.kernel_ridge import KernelRidge

ker = KernelRidge(alpha = 40)
ker.fit(htrain, ptrain)

In [ ]:
print(ker.score(htest, ptest))
pred_ker = ker.predict(htest)
mean_squared_error(pred_ker, ptest)

## Linear Model

In [ ]:
lm = LinearRegression(normalize = True, n_jobs = -1)

In [ ]:
training = housing_features.drop(housing_features.columns[drop_col], axis=1)
testing = htest_features.drop(housing_features.columns[drop_col], axis=1)

In [ ]:
print(training.shape)
print(testing.shape)

In [ ]:
htrain, htest, ptrain, ptest = train_test_split(training, housing.saleprice, test_size = 0.33, random_state = 0)

In [ ]:
lm.fit(htrain, ptrain)

lm.score(htest, ptest)

In [ ]:
lm.sample_

In [ ]:
pred = lm.predict(htest)
mean_squared_error(pred, ptest)

In [ ]:
pred = np.exp(lm.predict(testing))
print(pred[1:3])
Submission(htest_id, pred)